<a href="https://colab.research.google.com/github/RohaidAhmed/DR-Train-Dataset/blob/main/clahe_training_mobilnet1_.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

from google.colab import drive
drive.mount('/content/drive')

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
!unzip "/content/drive/MyDrive/network2/traintestvalid.zip" -d my_data

Streaming output truncated to the last 5000 lines.
  inflating: my_data/traintestvalid/train1/2/007-5243-300.jpg  
  inflating: my_data/traintestvalid/train1/2/007-5244-300.jpg  
  inflating: my_data/traintestvalid/train1/2/007-5245-300.jpg  
  inflating: my_data/traintestvalid/train1/2/007-5246-300.jpg  
  inflating: my_data/traintestvalid/train1/2/007-5247-300.jpg  
  inflating: my_data/traintestvalid/train1/2/007-5248-300.jpg  
  inflating: my_data/traintestvalid/train1/2/007-5249-300.jpg  
  inflating: my_data/traintestvalid/train1/2/007-5250-300.jpg  
  inflating: my_data/traintestvalid/train1/2/007-5251-300.jpg  
  inflating: my_data/traintestvalid/train1/2/007-5252-300.jpg  
  inflating: my_data/traintestvalid/train1/2/007-5254-300.jpg  
  inflating: my_data/traintestvalid/train1/2/007-5255-300.jpg  
  inflating: my_data/traintestvalid/train1/2/007-5256-300.jpg  
  inflating: my_data/traintestvalid/train1/2/007-5257-300.jpg  
  inflating: my_data/traintestvalid/train1/2/007-5258

In [3]:
from keras.applications import InceptionResNetV2
from keras.layers.core import Lambda
from keras.applications.mobilenet_v2 import MobileNetV2
from keras.applications.inception_v3 import InceptionV3 
from keras.applications.densenet import DenseNet121
from keras.applications.vgg16 import VGG16
from tensorflow.keras.applications.resnet50 import ResNet50
from keras.applications.xception import Xception
from keras.applications.mobilenet import MobileNet
from keras.layers import *
from keras.models import Model
from keras.callbacks import ModelCheckpoint, ReduceLROnPlateau
from keras.preprocessing.image import ImageDataGenerator
from keras.optimizers import *
# from tensorflow.python.keras.utils.multi_gpu_utils import multi_gpu_model
# from tensorflow.keras.utils.multi_gpu_utils import multi_gpu_model
from matplotlib import pyplot as plt
from keras.models import load_model
import keras.backend as K
import os
import tensorflow as tf
from sklearn.metrics import f1_score
from sklearn.model_selection import train_test_split

import numpy as np
import pandas as pd
from PIL import Image
import cv2 

import sys
import os.path

In [4]:

def Global_attention_block(inputs):
    shape=K.int_shape(inputs)
    x=AveragePooling2D(pool_size=(shape[1],shape[2])) (inputs)
    x=Conv2D(shape[3],1, padding='same') (x)
    x=Activation('relu') (x)
    x=Conv2D(shape[3],1, padding='same') (x)
    x=Activation('sigmoid') (x)
    C_A=Multiply()([x,inputs])
    
    x=Lambda(lambda x: K.mean(x,axis=-1,keepdims=True))  (C_A)
    x=Activation('sigmoid') (x)
    S_A=Multiply()([x,C_A])
    return S_A

In [5]:
def Category_attention_block(inputs,classes,k):
    shape=K.int_shape(inputs)
    F=Conv2D(k*classes,1, padding='same') (inputs)
    F=BatchNormalization() (F)
    F1=Activation('relu') (F)
    
    F2=F1
    x=GlobalMaxPool2D()(F2)
    
    x=Reshape((classes,k)) (x)
    S=Lambda(lambda x: K.mean(x,axis=-1,keepdims=False))  (x)
    
    x=Reshape((shape[1],shape[2],classes,k)) (F1)
    x=Lambda(lambda x: K.mean(x,axis=-1,keepdims=False))  (x)
    x=Multiply()([S,x])
    M=Lambda(lambda x: K.mean(x,axis=-1,keepdims=True))  (x)
    
    semantic=Multiply()([inputs,M])
    return semantic


In [6]:
def smooth_curve(points, factor=0.9):
    smoothed_points = []
    for point in points:
        if smoothed_points:
            previous = smoothed_points[-1]
            smoothed_points.append(previous * factor + point * (1 - factor))
        else:
            smoothed_points.append(point)
    return smoothed_points    
   

In [7]:
def plotmodel(history,name):
    
    acc = history.history['acc']
    val_acc = history.history['val_acc']
    loss = history.history['loss']
    val_loss = history.history['val_loss']
    epochs = range(1, len(acc) + 1) 
    
    plt.figure(1)                  
    plt.plot(epochs,smooth_curve(acc))
    plt.plot(epochs,smooth_curve(val_acc))
    plt.ylabel('acc')
    plt.xlabel('epoch')
    plt.legend(['train_acc', 'val_acc'], loc='upper left')
    plt.savefig('acc_'+name+'.png')
    
    plt.figure(2)
    plt.plot(epochs,smooth_curve(loss))
    plt.plot(epochs,smooth_curve(val_loss))
    plt.ylabel('loss')
    plt.xlabel('epoch')
    plt.legend(['train_loss', 'val_loss'], loc='upper right')
    plt.savefig('loss_'+name+'.png')

In [8]:
def label_smooth(y_true, y_pred):
    y_true=((1-0.1)*y_true+0.05)
    return K.categorical_crossentropy(y_true, y_pred) 
 

In [9]:
def get_base_model(model_name,image_size):
    if model_name =='vgg16':
        base_model=VGG16              (include_top=False,weights='imagenet',input_shape=(image_size,image_size,3))
    if model_name =='resnet50':
        base_model=ResNet50           (include_top=False,weights='imagenet',input_shape=(image_size,image_size,3))
    if model_name =='xception':
        base_model=Xception           (include_top=False, weights='imagenet',input_shape=(image_size,image_size,3))
    if model_name =='densenet121':
        base_model=DenseNet121       (include_top=False, weights='imagenet',input_shape=(image_size,image_size,3))
    if model_name =='mobilenet0.75':
        base_model=MobileNet         (include_top=False,weights='imagenet',alpha=0.75,input_shape=(image_size,image_size,3))
    if model_name =='mobilenet1.0':
        base_model=MobileNet         (include_top=False,weights='imagenet',alpha=1.0,input_shape=(image_size,image_size,3))
    if model_name =='mobilenetv2':
        base_model=MobileNetV2      (include_top=False,weights='imagenet',alpha=1.0,input_shape=(image_size,image_size,3))
    if model_name =='inceptionv3':   
        base_model=InceptionV3       (include_top=False,weights='imagenet',input_shape=(image_size,image_size,3))
    if model_name =='inceptionv2':
        base_model=InceptionResNetV2 (include_top=False, weights='imagenet',input_shape=(image_size,image_size,3))
    return base_model

In [10]:
# def get_data(folder):
#     X = []
#     y = []

#     folder_num = 0
#     for wbc_type in sorted(os.listdir(folder)):
#         if not wbc_type.startswith('.'):
#           subFolder = os.path.join(folder , wbc_type)
#           for image_filename in os.listdir(subFolder):
#               img_file = cv2.imread(subFolder + '/' + image_filename)
#               # print(folder + wbc_type)
#               if img_file is not None:
#                   img_arr = np.asarray(img_file)
#                   #img_arr = cv2.cvtcolor(img_arr, cv2.COLOR_BGR2RGB)
#                   # img_arr = cv2.resize(img_arr,(224,224))
#                   X.append(img_arr)
#                   y.append(folder_num)
#                   # print(wbc_type)
#       # print(folder_num)
#         folder_num = folder_num + 1
#     # X = np.asarray(X)
#     # X = X.astype ('float32')
#     # X /= 255
#     y = np.asarray(y)
#     return X,y

In [11]:
# import os
# import numpy as np
# import cupy as cp
# from PIL import Image

# def load_images(train_dir):
#     # Get the list of classes (subdirectories) in the train directory
#     classes = os.listdir(train_dir)

#     # Create empty lists for x_train and y_train
#     x_train = []
#     y_train = []

#     # Iterate over the classes
#     for class_index, class_name in enumerate(classes):
#         class_dir = os.path.join(train_dir, class_name)

#         # Get the list of image files in the class directory
#         image_files = os.listdir(class_dir)

#         # Iterate over the image files
#         for image_file in image_files:
#             image_path = os.path.join(class_dir, image_file)

#             # Load and resize the image using PIL
#             image = Image.open(image_path)
#             image = image.resize((image_size, image_size))

#             # Convert the image to a numpy array and add it to x_train
#             x_train.append(np.array(image))

#             # Add the class index to y_train
#             y_train.append(class_index)

#     # Convert x_train and y_train to Cupy arrays for GPU utilization
#     # x_train = cp.array(x_train)
#     # y_train = cp.array(y_train)

#     return x_train, y_train
  

In [21]:
from tensorflow.keras.preprocessing.image import load_img, img_to_array

def load_images(train_dir):
    # Get the list of classes (subdirectories) in the train directory
    classes = sorted(os.listdir(train_dir))

    x_train = []
    y_train = []

    # Iterate over the classes
    for class_index, class_name in enumerate(classes):
        class_dir = os.path.join(train_dir, class_name)

        # Get the list of image files in the class directory
        image_files = os.listdir(class_dir)

        # Iterate over the image files
        for image_file in image_files:
            image_path = os.path.join(class_dir, image_file)

            # Load and preprocess the image using TensorFlow
            image = load_img(image_path, target_size=(image_size, image_size))
            image_array = img_to_array(image)
            image_array = image_array / 255.0  # Normalize pixel values to the range [0, 1]

            # Append the preprocessed image and class index to x_train and y_train
            x_train.append(image_array)
            y_train.append(class_index)

    # Convert x_train and y_train to numpy arrays
    x_train = np.array(x_train)
    y_train = np.array(y_train)

    return x_train, y_train

In [13]:
def clahe(image):
    if image.dtype == np.uint16:
        image = (image / 256).astype(np.uint8)  # Convert 16-bit image to 8-bit
    elif image.dtype != np.uint8:
        image = image.astype(np.uint8)  # Convert other data types to 8-bit
    image = cv2.cvtColor(image, cv2.COLOR_RGB2LAB)
    clahe = cv2.createCLAHE(clipLimit=0.02, tileGridSize=(5,5))
    image[:,:,0] = clahe.apply(image[:,:,0])
    image = cv2.cvtColor(image, cv2.COLOR_LAB2RGB)
    return image

In [14]:
def train_model(model,train_data,total_images_train,valid_data,total_images_val,image_size,batch_size,save_name,lr1,lr2,Epochs1,Epochs2):
    
    
   
    
#     train_num=len(train_data)
#     valid_num=len(valid_data)

    earlystop = tf.keras.callbacks.EarlyStopping(monitor='val_loss',mode='min',patience=9,min_delta=0.001)
    rz = ReduceLROnPlateau(monitor='val_loss',mode='min',patience=5,min_lr=1e-9)
    check = ModelCheckpoint('updated/bestModel.h5',verbose=1,save_best_only=True,save_weights_only=True)
    callbacks = [rz, earlystop, check]
 
    
    for layer in base_model.layers:
        layer.trainable = True
        
    model.compile(optimizer=Adam(learning_rate=lr2,decay=0.00001),loss=loss_fun,metrics=['acc'])
    history=model.fit(train_data,
                        steps_per_epoch=total_images_train/batch_size,
                        validation_data=valid_data,
                        validation_steps=total_images_val/batch_size,
                        epochs=Epochs2,
                        workers=2,
                        callbacks=callbacks)
 
    
    
    return history

In [15]:
train_dir = '/content/my_data/traintestvalid/train'
test_dir = '/content/my_data/traintestvalid/test'
val_dir = '/content/my_data/traintestvalid/val'

In [16]:
# xtrain, ytrain_org = get_data(train_dir)
# xtest, ytest_org = get_data(test_dir)

# xval, yval_org = get_data(val_dir)

In [23]:
os.environ["CUDA_VISIBLE_DEVICES"] = "0"    
loss_fun= 'categorical_crossentropy'  
gpu_num=1
k=5
lr1=0.005
lr2=0.001
batch_size= 16
image_size=512
classes=5


# xtrain = np.array(xtrain, dtype="float32")
# xtrain /=255

# xtest = np.array(xtest, dtype="float32")
# xtest /=255







In [ ]:
xtrain, ytrain_org = load_images(train_dir)


In [ ]:
ytrain = np.array(ytrain_org)
ytrain = tf.keras.utils.to_categorical(ytrain, classes)

In [ ]:
xtest, ytest_org = load_images(test_dir)


In [ ]:
ytest = np.array(ytest_org)
ytest = tf.keras.utils.to_categorical(ytest, classes)

In [ ]:
base_model=get_base_model('mobilenet1.0',image_size)  
base_in=base_model.input
base_out=base_model.output

x=Global_attention_block(base_out)
base_out=Category_attention_block(x,classes,k)

x=GlobalAveragePooling2D()(base_out)
out=Dense(classes,activation='softmax')(x)

if gpu_num>1:
    model=Model(base_model.input,out)
    #model.summary()
    parallel_model = multi_gpu_model(model, gpus=gpu_num)
    parallel_model.summary()
else:
    parallel_model=Model(base_model.input,out)
    parallel_model.summary()


In [ ]:
# 

In [ ]:
# # import tensorflow as tf
# # import os

# data_dir = r'C:\Users\VC-Tech Server 2\Documents\New folder\Network3\eyepacs\DATASETS__resize_split\train'
class_names = ['0', '1', '2','3','4']  

# # Define the image data generator
# image_size = (512, 512)
# # batch_size = 32

datagen = tf.keras.preprocessing.image.ImageDataGenerator(
#     rescale=1.0/255.0,
    preprocessing_function=clahe,horizontal_flip=True,vertical_flip=True,rotation_range=90
#     validation_split=0.2  # Split 20% of the images for validation
)

# Load the dataset using the image data generator
train_data = datagen.flow_from_directory(
    train_dir,
    target_size=(image_size,image_size),
    batch_size=batch_size,
    class_mode='categorical',
    shuffle=True,
    subset='training'
)

# Create a dictionary to store the number of images for each class
class_counts = {class_name: 0 for class_name in class_names}

# Count the number of images for each class
for class_name in class_names:
    class_dir = os.path.join(train_dir, class_name)
    num_images = len(os.listdir(class_dir))
    class_counts[class_name] = num_images

    print(f"Class: {class_name}, Number of images: {num_images}")

# Additional code to display the total number of images
total_images_train = sum(class_counts.values())
print(f"Total number of images: {total_images_train}")


In [ ]:
# # import tensorflow as tf
# # import os

# test_dir = r'C:\Users\VC-Tech Server 2\Documents\New folder\Network3\eyepacs\DATASETS__resize_split\test'
# # class_names_test = ['0', '1', '2','3','4']  

datagen_test = tf.keras.preprocessing.image.ImageDataGenerator(
    rescale=1.0/255.0,
#     validation_split=0.2  # Split 20% of the images for validation
)

# Load the dataset using the image data generator
test_data = datagen_test.flow_from_directory(
    test_dir,
    target_size=(image_size,image_size),
    batch_size=batch_size,
    class_mode='categorical'
)

# Create a dictionary to store the number of images for each class
class_counts_test = {class_name: 0 for class_name in class_names}

# Count the number of images for each class
for class_name in class_names:
    class_dir_test = os.path.join(test_dir, class_name)
    num_images_test = len(os.listdir(class_dir_test))
    class_counts_test[class_name] = num_images_test

    print(f"Class: {class_name}, Number of images: {num_images_test}")

# Additional code to display the total number of images
total_images_test = sum(class_counts_test.values())
print(f"Total number of images: {total_images_test}")


In [ ]:
# # import tensorflow as tf
# # import os

# val_dir = r'C:\Users\VC-Tech Server 2\Documents\New folder\Network3\eyepacs\DATASETS__resize_split\val'
# # class_names_test = ['0', '1', '2','3','4']  


xval, yval_org = load_images(val_dir)
yval = np.array(yval_org)
yval = tf.keras.utils.to_categorical(yval, classes)




In [ ]:

datagen_val = tf.keras.preprocessing.image.ImageDataGenerator(
    # rescale=1.0/255.0,
    
#     validation_split=0.2  # Split 20% of the images for validation
)

# Load the dataset using the image data generator
valid_data = datagen_test.flow_from_directory(
    val_dir,
    target_size=(image_size,image_size),
    batch_size=batch_size,
    class_mode='categorical'
)

# Create a dictionary to store the number of images for each class
class_counts_val = {class_name: 0 for class_name in class_names}

# Count the number of images for each class
for class_name in class_names:
    class_dir_val = os.path.join(val_dir, class_name)
    num_images_val = len(os.listdir(class_dir_val))
    class_counts_val[class_name] = num_images_val

    print(f"Class: {class_name}, Number of images: {num_images_val}")

# Additional code to display the total number of images
total_images_val = sum(class_counts_val.values())
print(f"Total number of images: {total_images_val}")

In [ ]:

# # traincsv = r'C:\Users\VC-Tech

# # trainx1,valx,trainy1,valy=train_test_split(id_code,diagnosis,test_size=0.2)
# # trainx,testx,trainy,testy=train_test_split(trainx1,trainy1,test_size=0.25)





# csv_file = r'C:\Users\VC-Tech Server 2\Documents\New folder\Network3\DR_grading.csv'
# data_dir = r'C:\Users\VC-Tech Server 2\Documents\New folder\Network3\DR_grading\DR_grading'
# df = pd.read_csv(csv_file)

# id_code = df['id_code'].values
# df["diagnosis"] = df["diagnosis"].astype(str)

# diagnosis = df['diagnosis']

# traindata,testdata1=train_test_split(df,test_size=0.4)
# testdata,validdata =train_test_split(testdata1,test_size=0.5)
# # x,trainy1,valy=train_test_split(id_code,diagnosis,test_size=0.2)
# # trainx,testx,trainy,testy=train_test_split(trainx1,trainy1,test_size=0.25)

# # trainx, X_rem, trainy, y_rem = train_test_split(id_code,diagnosis, train_size=0.6)

# # # Now since we want the valid and test size to be equal (20% each of overall data). 
# # # we have to define valid_size=0.5 (that is 50% of remaining data)
# # test_size = 0.5
# # valx, testx, valy, testy = train_test_split(X_rem,y_rem, test_size=0.5)



# train=ImageDataGenerator(preprocessing_function=clahe,horizontal_flip=True,vertical_flip=True,rotation_range=90)          
# valid = ImageDataGenerator(preprocessing_function= clahe)


# train_data=train.flow_from_dataframe(dataframe=traindata,
# #                                      df[df['id_code'].isin(trainx)],
#                                      directory=data_dir,
#                                      x_col='id_code',
#                                      y_col='diagnosis',
#                                      target_size=(image_size,image_size),
#                                      batch_size=batch_size,
#                                      shuffle=True,
#                                      class_mode='categorical')
# valid_data=valid.flow_from_dataframe(dataframe=validdata,
# #                                      df[df['id_code'].isin(valx)],
#                                      directory=data_dir,
#                                      x_col='id_code',
#                                      y_col='diagnosis',
#                                      target_size=(image_size,image_size),
#                                      batch_size=batch_size,
#                                      shuffle=False,
#                                      class_mode='categorical')

In [ ]:
# with tf.device('/GPU:0'):
history=train_model(parallel_model,train_data,total_images_train,valid_data,total_images_val,image_size,batch_size,'mobilenet1.0',lr1,lr2,1,70)



In [ ]:
plotmodel(history,'mobilenet1.0')

In [ ]:
from tensorflow.keras import models
parallel_model.load_weights("updated/bestModel.h5")

In [ ]:
# test = ImageDataGenerator(rescale=1./255)
# test_data=test.flow_from_dataframe(dataframe=testdata,
# #                                    df[df['id_code'].isin(testx)],
#                                  directory=data_dir,
#                                  x_col='id_code',
#                                  y_col='diagnosis',
#                                  target_size=(image_size,image_size),
#                                  batch_size=batch_size,
#                                  shuffle=False,
#                                  class_mode='categorical')

In [ ]:
predictions = parallel_model.predict(test_data,  verbose=1)

In [ ]:
from sklearn.metrics import  precision_score, recall_score, accuracy_score,classification_report ,confusion_matrix
#geting predictions on test dataset
y_pred = np.argmax(predictions, axis=1)
targetnames = ['0', '1','2','3','4']
#getting the true labels per image 
y_true = test_data.classes
#getting the predicted labels per image 
y_prob=predictions
from tensorflow.keras.utils import to_categorical
y_test = to_categorical(y_true)

# Creating classification report 
report = classification_report(y_true, y_pred, target_names=targetnames)

print("\nClassification Report:")
print(report)